## Environment

Run the appropriate "environment" code before running any solutions.

In [8]:
#@title environment: google.colab
from google.colab import files
fread = lambda path: open(path,'r').read()

In [7]:
#@title environment: vscode
fread = lambda path: open('inputs/'+path, 'r').read()

## Imports

Import general-purpose libraries before running any solutions

In [16]:
import re, math
from operator import add, sub
from functools import reduce
from math import prod
from itertools import repeat

sign = lambda n: int(math.copysign(1, n))

class formats:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    ORANGE = '\033[93m'
    RED = '\033[91m'
    WHITE = '\033[0m'
    BOLD = '\033[1m'

# Solutions

Solution inputs should be uploaded with the form `23-{dayNumber}.txt` (e.g., the input for Day 5 would be uploaded as `23-5.txt`). Each solution has the input file name hard-coded into the script, so if you want to run the script against a test file you will need to edit the file name.

## Week 1

In [ ]:
#@title Day 1: Trebuchet?!
ls = fread('23-1.txt').splitlines()

print(sum(map(lambda d: int(d[0]+d[-1]),[[c for c in l if '/' < c < ':'] for l in ls])))

ns = {n: n[0]+str(i)+n[-1] for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))}
[ls:= [l.replace(k,ns[k]) for l in ls] for k in ns]
print(sum([int(re.search('\\d',l).group()+re.search('\\d',l[::-1]).group()) for l in ls]))


54573
54591


In [ ]:
#@title Day 2: Cube Conundrum
maxes = {'red': 12, 'green': 13, 'blue': 14}

ls = fread('23-2.txt').splitlines()
gs = [[{p.strip().split(' ')[1]: int(p.strip().split(' ')[0])
        for p in r.split(',')}
        for r in l.split(':')[1].split(';')]
        for l in ls]

valid = [i+1 for (i,g) in enumerate(gs)
             if all([all([maxes[k]>=r[k] for k in r]) for r in g])]
print(sum(valid))

powers = [math.prod([max([r.get(k,0) for r in g]) for k in maxes]) for g in gs]
print(sum(powers))

2727
56580


In [ ]:
#@title Day 3: Gear Ratios
f = fread('23-3.txt').splitlines()

special = lambda x,y,n: [re.search('[^\\d.]',f[min(max(yi,0),len(f)-1)][max(x-1,0):min(x+n+1,len(f[0])-1)]) != None for yi in range(y-1,y+2)]
ps = {(m.start(),y): m.group() for (y,l) in enumerate(f) for m in re.finditer('\\d+',l)}
ns = [int(ps[(x,y)]) for (x,y) in ps if any(special(x,y,len(ps[(x,y)])))]
print(sum(ns))

stars = lambda x,y,n: [
  (m.start(),yi)
  for yi in range(max(0,y-1),min(y+2,len(f)))
  for m in re.finditer('\\*', f[yi]) if (x-2)<m.start()<(x+n+1)
]

ss = [(s, int(ps[(x,y)])) for (x,y) in ps for s in stars(x,y,len(ps[(x,y)]))]
pgs = [[n for t,n in ss if t==u] for u in set([s for s,n in ss])]
gs = [pg for pg in pgs if len(pg)==2]
print(sum([math.prod(g) for g in gs]))


529618
77509019


In [ ]:
#@title Day 4: Scratchcards
f = fread('23-4.txt').splitlines()

cs = [[set([int(n) for n in m.group().split(' ') if n != ''])
  for m in re.finditer('[\\d ]+', l.split(':')[1])]
  for l in f]

wins = [sum([1 for n in c[1] if n in c[0]]) for c in cs]
score = sum([1<<(w-1) for w in wins if w > 0])
print(score)

cards = [1]*len(wins)
[cards:= [sum(n) for n in zip(cards,
  ([0]*(i+1))+([cards[i]]*w)+([0]*(len(wins)-w-i-1)))]
  for i,w in enumerate(wins)]
print(sum(cards))

18619
8063216


In [ ]:
#@title Day 5-1: If You Give A Seed A Fertilizer
f = [s.splitlines() for s in fread('23-5.txt').split('\n\n')]

fn = lambda q,p,r: lambda n: (n-p)+q if p <= n < p + r else None
seeds = list(map(int, f[0][0].split(' ')[1:]))
maps = [[fn(*map(int, l.split(' ')))
  for l in s[1:]]+[lambda n: n]
  for s in f[1:]]

applyone = lambda n, fns: next(g(n) for g in fns if g(n) != None)

def apply(n):
  result = n
  [result := applyone(result, m) for m in maps]
  return result

print(min(apply(n) for n in seeds))

836040384


In [ ]:
#@title Day 5-2: If You Give A Seed A Fertilizer
f = [s.splitlines() for s in fread('23-5.txt').split('\n\n')]

class Range:
  def __init__(self, start, end):
    self.start = start
    self.end = end
    self.length = end - start

class AlmanacRange(Range):
  def __init__(self, dest, source, length):
    Range.__init__(self, source, source + length)
    self.__op = lambda n: n + (dest - source)

  def apply(self, r):
    return (
        (set() if self.start <= r.start else {Range(r.start, min(r.end, self.start))})
        .union(set() if self.end >= r.end else {Range(max(r.start, self.end), r.end)}),
        set() if self.end < r.start or r.end < self.start else
        {Range(self.__op(max(self.start, r.start)), self.__op(min(self.end, r.end)))}
        )

class AlmanacMap:
  def __init__(self, name, ranges):
    self.__name = name
    self.__ranges = ranges

  def apply(self, rs):
    acc = (set(rs),{})
    [acc := reduce(
          lambda a,b: (a[0].union(b[0]), a[1].union(b[1])),
          [amr.apply(r) for r in acc[0]] + [(set(),acc[1])])
          for amr in self.__ranges]

    return acc[0].union(acc[1])

class Almanac:
  def __init__(self, maps):
    self.__maps = maps

  def parse(lines):
    return Almanac([
      AlmanacMap(s[0], [AlmanacRange(*map(int, l.split(' '))) for l in s[1:]])
      for s in lines])

  def apply(self, rs):
    acc = set(rs)
    [acc := m.apply(acc) for m in self.__maps]
    return acc

ps = list(map(int, f[0][0].split(' ')[1:]))
seeds = [Range(ps[i], ps[i]+ps[i+1])
    for i in range(0,len(ps),2)]

almanac = Almanac.parse(f[1:])
best = min([r.start for r in almanac.apply(seeds)])
print(best)

10834440


In [ ]:
#@title Day 6: Wait For It
f = fread('23-6.txt').splitlines()

solve = lambda t0,t1: [math.ceil((0.5*t0) + (s*0.5*math.sqrt((t0*t0)-(4*t1)))) for s in [1,-1]]
solutions = lambda rs: prod([reduce(lambda a,b: a-b, list(map(math.ceil, solve(*r)))) for r in rs])

races = list(zip(*[map(int, re.split('[ ]+', l)[1:]) for l in f]))
print(solutions(races))

race = tuple(int(l.replace(' ','').split(':')[1]) for l in f)
print(solutions([race]))

800280
45128024


In [ ]:
#@title Day 7: Camel Cards
f = fread('23-7.txt')
[f := f.replace(*c) for c in ['T:','Kk','Az']]

# Helpers
get_type = lambda h: 5 * (max(h.count(c) for c in set(h))) - len(set(h))
get_joker_type = lambda h: get_type(h.replace('J', max(set(h), key=lambda c: (c != 'J', h.count(c)))))
winnings = lambda hs: sum((i + 1) * h[1] for i, h in enumerate(hs))

# Part 1
hands = [[l[0:5],int(l[6:])] for l in f.splitlines()]
hands.sort(key=lambda h: (get_type(h[0]), h[0]))
print(winnings(hands))

# Part 2
hands.sort(key=lambda h: (get_joker_type(h[0]), h[0].replace('J','!')))
print(winnings(hands))

251136060
249400220


## Week 2

In [ ]:
#@title Day 8: Haunted Wasteland
from itertools import count
from math import lcm, gcd

f = fread('23-8.txt').splitlines()

graph = {n[0]: n[1:] for n in [[m.group() for m in re.finditer('[A-Z]+', l)] for l in f[2:]]}
walk = lambda u: next(i for i in count() if (u := graph[u][f[0][i%len(f[0])]=='R']) == 'ZZZ') + 1
print(walk('AAA'))

walkToZ = lambda curr: next(i for i in count() if (curr := graph[curr][f[0][i%len(f[0])]=='R'])[-1] == 'Z') + 1
print(lcm(*[walkToZ(k) for k in graph if k[-1] == 'A']))

20513
15995167053923


In [ ]:
#@title Day 9: Mirage Maintenance
f = [[int(n) for n in l.split(' ')] for l in fread('23-9.txt').splitlines()]

deltas = lambda l: [l[i]-l[i-1] for i in range(1,len(l))]
predict = lambda l: l[-1] if set(deltas(l)) == {0} else predict(deltas(l))+l[-1]
print(sum([predict(l) for l in f]))

history = lambda l: l[0] if set(deltas(l)) == {0} else l[0]-history(deltas(l))
print(sum([history(l) for l in f]))

1666172641
933


In [ ]:
#@title Day 10: Pipe Maze
import sys

f = fread('23-10.txt').splitlines()

start = next((col, row) for row,l in enumerate(f) for col,c in enumerate(l) if c == 'S')
connects = '-J7 7|F -LF J|L'.split(' ')

def get_neighbors (u):
  c = f[u[1]][u[0]]
  x,y = u
  ns = [] + [[None],[(x+1,y)]][c in set('S-LF')] \
    + [[None],[(x,y-1)]][c in set('S|JL')] \
    + [[None],[(x-1,y)]][c in set('S-J7')] \
    + [[None],[(x,y+1)]][c in set('S|7F')]

  ns = {n for i,n in enumerate(ns)
    if n != None and n[0] < len(f[0]) and n[1] < len(f) and f[n[1]][n[0]] in set(connects[i])}
  return ns

seen = set()
distances = {start: 0}
to_visit = {start}

while(len(to_visit) > 0):
  u = min(to_visit, key=lambda n: distances[n])
  to_visit.remove(u)

  neighbors = get_neighbors(u)

  for v in neighbors:
    distances[v] = min(distances.get(v,sys.maxsize), distances[u]+1)
  seen = seen.union({u})
  to_visit = to_visit.union(neighbors.difference(seen))

print(max(distances[k] for k in distances))

from itertools import groupby

f = [''.join([c if (col,row) in distances and distances[(col,row)] < sys.maxsize else '.' for col,c in enumerate(l)]) for row,l in enumerate(f)]

ks = sorted(distances, key=lambda k: (k[1],k[0]))
ks = [list(v) for k,v in groupby(ks, lambda k: k[1])]
#print(ks)

filled = 0
for l in f:
  #print(l,end=' => ')
  l = l.replace('S','|')
  l = l.replace('-','')
  l = l.replace('F7','')
  l = l.replace('FJ','|')
  l = l.replace('L7','|')
  l = l.replace('LJ','')
  #print(l)

  filling = False
  for c in l:
    if c == '|': filling = not filling
    else: filled += int(filling)

print(filled)

6786
495


In [ ]:
#@title Day 11: Cosmic Expansion
from itertools import repeat

f = fread('23-11.txt').splitlines()

expand = ({x for x in range(len(f[0]))[::-1] if all([l[x]=='.' for l in f])},
          {y for y in range(len(f))[::-1] if re.match('\.*$', f[y])})
galaxies = [(x,y) for y,l in enumerate(f) for x,c in enumerate(l) if c == '#']

def get_distance(g,h,n):
  return sum([
      abs(g[i]-h[i]) + (n-1)*len(expand[i].intersection(range(g[i],h[i],sign(h[i]-g[i]))))
      for i in [0,1]])

tally = lambda n: sum([get_distance(g,h,n) for i,g in enumerate(galaxies) for h in galaxies[i+1:]])

print(tally(2))
print(tally(1000000))

9543156
625243292686


In [ ]:
#@title Day 12: Hot Springs
import numpy as np

f = [l.split(' ') for l in fread('23-12.txt').splitlines()]

def check(s, d):
  cache = [np.zeros(len(d)+1) for i in range(len(s)+1)]
  cache[0][0] = 1

  for i in range(1,len(s)+1):
    c = s[i-1]

    for j in range(0, len(d)+1):
      if c in set('.?'):
        cache[i][j] += cache[i-1][j]

      if j > 0 and i >= d[j-1]:
        i_start = i-d[j-1]
        if all(c in set('#?') for c in s[i_start:i]) and (i_start == 0 or s[i_start-1] in set('.?')):
          cache[i][j] += cache[max(0,i_start-1)][j-1]

  return cache[len(s)][len(d)]

def tally(ls, vs): return sum([check(p, vs[i]) for i,p in enumerate(ls)])

springs = [l[0] for l in f]
damages = [list(map(int,l[1].split(','))) for l in f]
print(tally(springs,damages))

springs = ['?'.join(repeat(p,5)) for p in springs]
damages = [list(chain(*repeat(n,5))) for n in damages]
print(tally(springs,damages))

6949.0
51456609952403.0


In [ ]:
#@title Day 13: Point of Incidence
f = [p.splitlines() for p in fread('23-13.txt').replace('#','1').replace('.','0').split('\n\n')]

maps = list(zip(
    [list(map(lambda l: int(l,2), ls)) for ls in f],
    [list(map(lambda x: int(''.join(l[x] for l in ls),2), range(len(ls[0])))) for ls in f]
))

def is_symmetric(n, ls, smudge=1):
  ct = 0
  for i in range(0,min(n,len(ls)-n)):
    if ls[n-i-1] == ls[n+i]: continue
    if ct == smudge: return False
    if '{0:b}'.format(ls[n-i-1]^ls[n+i]).strip('0') != '1': return False
    ct += 1
  return ct == smudge

hs = [i for m in maps for i in range(1,len(m[0])) if is_symmetric(i,m[0],smudge=0)]
vs = [i for m in maps for i in range(1,len(m[1])) if is_symmetric(i,m[1],smudge=0)]
print(sum(hs)*100 + sum(vs))

hs = [i for m in maps for i in range(1,len(m[0])) if is_symmetric(i,m[0],smudge=1)]
vs = [i for m in maps for i in range(1,len(m[1])) if is_symmetric(i,m[1],smudge=1)]
print(sum(hs)*100 + sum(vs))

32723
34536


## Latest

In [84]:
#@title Day 14: Parabolic Reflector Dish
f = fread('23-14.txt').splitlines()

# Part 1
rotate = lambda m,direction=1: zip(*m[::direction])
tally = lambda p: sum((i+1)*l.count('O') for i,l in enumerate(p[::-1]))

slide_left = lambda p: [''.join(
  ['O' * chunk.count('O') + '.' * chunk.count('.') + '#'
  for chunk in ''.join(l).split('#') ])[:-1]
  for i,l in enumerate(p)]

print(tally(list(rotate(slide_left(rotate(f))))))

# Part 2
log = lambda p: '\n'.join(''.join(l) for l in p)

def cycle(p):
  n = slide_left(rotate(p))
  w = slide_left(rotate(n))
  s = slide_left(rotate(w,-1))
  e = slide_left(rotate(s,-1))
  return list(map(lambda l: l[::-1], e[::-1]))

def project(curr, n):
  states = {}
  steps = [curr]

  for i in range(1,n+1):
    steps.append(cycle(steps[i-1]))
    s = log(steps[i])

    if(s in states):
      last = states[s]
      cycle_length = i - last
      return steps[((n - last) % cycle_length) + last]

    states[s] = i
  return curr

print(tally(project(f, 1000000000)))

113424
96003
